In [28]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [207]:
#determine whether the matrix is unitary over F_q
#we define conjugation by working over F_q^2, and using \alpha: x |--> x^q, an order two automorphism
#if q=p^r, alpha is the r^th power of Frobenius
def alpha(x,q):
    return x**q

In [208]:
#define the conjugate transpose of a matrix A by first computing the size of the base field, F_q
#if q is square, then q' = sqrt{q} so we are working over F_(q')^2 where q = (q')^2
def conjugate_transpose_pos_char(A):
    assert A.nrows() == A.ncols()
    n = A.nrows()
    field_size = A.base_ring().order()
    if field_size.is_square():
        q = sqrt(field_size)
    else:
        q = field_size
    return matrix([[alpha(A[i][j],q) for i in range(n)] for j in range(n)]).transpose()

In [255]:
#find the change-of-basis matrix M for which A = M*GS where GS is the Gram-Schmidt orthonormal basis of A
def unitary_change_of_basis(SGA,partition):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    if SGA.characteristic() > 0:
        P = SGA.base_ring()(1/group_size)*sum(rho(g)*conjugate_transpose_pos_char(rho(g)).transpose() for g in SGA.group())
        q = P.base_ring().cardinality()
        D, M = P.diagonalization()
        sqrt_D = diagonal_matrix([D[i][i].square_root(extend=True) for i in range(D.ncols())])
        return M*sqrt_D*M.inverse()
    else:
        P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
        return P.principal_square_root()

In [231]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition)
        unitary_factor = sqrt(specht_module.dimension()/SGA.group().cardinality())
        return unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [34]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [57]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    return matrix([flatten([hat(delta(g),partition,SGA,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in G]).transpose()

In [ ]:
#is the SGA DFT unitary?

In [107]:
SGA = SymmetricGroupAlgebra(GF(7),3)

In [108]:
A = SGA.dft(); A

[1 1 1 1 1 1]
[1 4 6 3 3 4]
[0 6 0 6 1 1]
[0 1 0 6 1 6]
[1 3 1 3 3 3]
[1 6 6 1 1 6]

In [109]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
A*A.transpose()

[6 0 0 0 0 0]
[0 3 0 0 0 0]
[0 0 4 0 0 0]
[0 0 0 4 0 0]
[0 0 0 0 3 0]
[0 0 0 0 0 6]

In [144]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [150]:
specht_module = SGA.specht_module(partition)

In [151]:
rho = specht_module.representation_matrix

In [152]:
P = SGA.base_ring()(1/SGA.group().cardinality())*sum(rho(g)*conjugate_transpose_pos_char(rho(g)).transpose() for g in SGA.group()); P

[6 3]
[3 6]

In [239]:
q = P.base_ring().cardinality()

In [275]:
F=GF(q**2)

In [259]:
F(3).square_root()

2*z2 + 6

In [273]:
GF(7**2)(3).square_root()

2*z2 + 6

In [225]:
D, M = P.diagonalization(); D

[3 0]
[0 2]

In [276]:
sqrt_D = diagonal_matrix([F(D[i][i]).square_root() for i in range(D.ncols())]); sqrt_D[0][0]

2*z2 + 6

In [277]:
M*D*M.inverse() == P

True

In [278]:
sqrt_D*sqrt_D == D

True

In [279]:
M*sqrt_D*M.inverse()

[  z2 + 5 6*z2 + 6]
[6*z2 + 6   z2 + 5]

In [280]:
(M*sqrt_D*M.inverse())**2

[6 3]
[3 6]

In [230]:
sqrt(specht_module.dimension()/SGA.group().cardinality())

sqrt(1/3)

In [234]:
U_dft = unitary_dft(SGA).simplify_full(); U_dft

AttributeError: 'FiniteField_prime_modn_with_category' object has no attribute 'size'

In [106]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()).simplify_full() == identity_matrix(SGA.group().cardinality())

True

In [ ]:
eigs = matrix(CDF,unitary_dft(3).simplify_full()).eigenvalues()

In [ ]:
eigs

In [ ]:
#what are the eigenvalues?
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))